## Chaos Base Class definition

In [86]:
import numpy as np

class ChaosGen () :    
    def __init__ (self, *args) :
        if len(args) > 0 :
            self.vec = np.random.random_sample (args[0])
            self.shape = args[0]
        else :
            self.vec = None
            self.shape = None
            
    def getIterate (self) :
        return self.vec
                 
    def chaosPoints (self, steps=1) :
        ret = self.vec
        self.evolveSteps (steps)
        return ret
    
    def evolve (self) :
        ''' Defined in the specific chaotic classes '''
        pass
    
    def evolveSteps (self, steps) :
        for i in range(0, steps) :
            self.evolve ()

## Logistic Class

In [87]:
class Logistic (ChaosGen) :
    def __init__ (self, shape, r) :
        ChaosGen.__init__(self, shape)
        self.r = r
        
    def evolve (self) :
        self.vec = self.r*self.vec*(1-self.vec)

## Tent Class

In [88]:
class Tent (ChaosGen) :
    def __init__ (self, shape, mu) :
        ChaosGen.__init__(self, shape)
        self.mu = mu
        
    def evolve (self) :
        self.vec = np.where (self.vec <= self.mu, self.vec/self.mu, (1-self.vec)/(1-self.mu))

## Lorenz Class

In [89]:
from scipy.integrate import odeint

class Lorenz (ChaosGen) :
    def lorenz (X, t, sigma, beta, rho) :
        x, y, z = X
        dXdt = [sigma*(y-x), x*(rho-z) - y, x*y - beta*z]
        return dXdt
    
    def __init__ (self, shape, params) :
        ChaosGen.__init__(self)
        self.shape = shape
        self.params = params
        
        t = np.linspace (0, 9999, 999999)
        sol = odeint (Lorenz.lorenz, np.random.rand(3), t, args = params)
        self.lims = np.array ([
                            [np.min(sol[:,0]), np.max(sol[:,0])],
                            [np.min(sol[:,1]), np.max(sol[:,1])],
                            [np.min(sol[:,2]), np.max(sol[:,2])]
                                ])
        
        self.state = np.empty (shape + (3,))
        self.vec = np.random.random_sample (shape)
        
        scaleState = (lambda mn, mx, st=np.random.random_sample (shape) : mn + (mx - mn)*st)
        self.state[...,0] = scaleState (self.lims[0,0], self.lims[0,1], self.vec)
        self.state[...,1] = scaleState (self.lims[1,0], self.lims[1,1])
        self.state[...,2] = scaleState (self.lims[2,0], self.lims[2,1])
        
        
    def evolveSteps (self, steps) :
        eps = 1e-5
        self.state = np.array ([\
                odeint(Lorenz.lorenz, self.state[pt], np.arange(0,0.01*(steps+1),0.01), args=self.params)[-1]\
                for pt in np.ndindex(self.shape)\
                ]).reshape(self.shape + (3,))
        
        self.vec = (lambda n2 : np.where (n2 > 1, 1-eps, n2))(\
                (lambda n1 : np.where (n1 < 0, eps, n1))(\
                (lambda st, mn, mx : (st - mn)/(mx - mn))(self.state[...,0], self.lims[0][0], self.lims[0][1])\
                ))
        
        
    def evolve (self) :
        self.evolveSteps (1)

## Initialising Chaotic Generators

In [93]:
chaos1 = Lorenz ((25,30),(10, 8.0/3, 28))
chaos2 = Tent ((25,30),0.49999)
chaos3 = Logistic ((25,30),4)

## Generating Chaotic Numbers

In [102]:
print ("Lorenz = \n" + str(chaos1.chaosPoints()))
print ("Tent = \n" + str(chaos2.chaosPoints()))
print ("Logistic = \n" + str(chaos3.chaosPoints()))

Lorenz = 
[[1.00000000e-05 5.73440193e-01 1.13200908e-02 6.25770067e-01
  6.59885415e-01 5.50491660e-01 7.78017818e-01 1.00000000e-05
  7.36099880e-01 8.05293408e-01 6.87904548e-01 3.42909253e-01
  7.57064604e-01 5.53027115e-01 8.05196806e-01 8.03375052e-01
  1.00000000e-05 4.25080096e-01 6.36217138e-01 5.01976348e-01
  3.63594950e-01 7.61276813e-01 1.36071972e-01 7.81919389e-01
  3.50803021e-01 4.96037050e-01 1.00000000e-05 6.66026428e-01
  4.39200966e-01 8.18390481e-01]
 [4.63379012e-01 5.76560784e-01 4.52793264e-01 1.00000000e-05
  9.89300221e-02 6.63067285e-01 1.03474706e-01 5.08066393e-01
  2.39180485e-01 7.44063440e-01 5.99921954e-01 4.60328844e-01
  2.74075317e-01 8.20052527e-01 6.72748786e-01 5.31509996e-01
  3.43681162e-01 1.00000000e-05 5.44563425e-01 6.08805454e-01
  7.44837101e-01 2.40919545e-01 8.23921844e-01 6.97496535e-01
  5.84013346e-01 4.78323906e-01 7.45406165e-01 5.46100298e-01
  2.18637386e-01 2.44611070e-01]
 [8.12390205e-01 3.16179068e-01 3.65532831e-01 3.1624398